<a href="https://colab.research.google.com/github/Sandwhaletree/Mai_Finlab/blob/main/%E8%82%A1%E7%A5%A8%E8%B2%A1%E5%8B%99%E5%88%86%E6%9E%90/A01_Crawing_publicinfo_%E5%85%AC%E5%8F%B8%E4%BB%A3%E7%A2%BC%E8%88%87%E5%85%AC%E5%8F%B8%E8%B3%87%E8%A8%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **🔥公開資訊-快速查詢公司代碼&公司資訊**

In [1]:
import requests
import pandas as pd
import datetime

In [13]:
def company_info(): #公司資訊

    #設計4個空變數，之後要呼叫4張公司資料表
    df1 = []  #company_info_sii
    df2 = []  #company_info_otc
    df2 = []  #company_info_rotc
    df4 = []  #company_info_pub

    #產生4種市場之公司基本資料
    markets = ["sii","otc","rotc","pub"] #sii=上市 otc=上櫃 rotc=興櫃 pub公開發行

    for market in markets:
      res =requests.get(f"https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK={market}&step=1&firstin=1") #透過網址把抓到的資料放到res
      df = pd.read_html(res.text)[0] #把需要的資料放在df
      df.columns = df.columns.str.replace(' ', '') #將公司 代號，去除空格，成為公司代號
      filt = df[df['公司代號']!= '公司 代號'] #利用欄位塞選，濾掉重複性的標題
      filt.insert(1,'市場別',market)
      # filt.set_index('公司代號', inplace=True) #將公司代號欄位，直接設計成index

      #把資料分別儲存至空變數中
      if market == "sii":
        df1 = filt
      elif market == "otc":
        df2 = filt
      elif market == "rotc":
        df3 = filt
      else:
        df4 = filt

    #建立空字典
    result={}

    #讀取df1~df4，並取出需要之資料
    for df in [df1,df2,df3,df4]:
      for index,col in df.iterrows():
        #key: value1、value2...、value2...
        result[col['公司代號']]=(col['營利事業統一編號'],col['公司簡稱'],col['市場別'],col['產業類別'],col['公司網址'])
    # print(result) #檢查1

    return result

In [3]:
def company_id (keys,result): #公司代碼
  # 建立空清單來存放結果
  values = []

  # 遍歷清單中的所有鍵值
  for key in keys:
      # 使用get方法查詢字典中的值
      value = result.get(key)
      # 將值加入結果清單中
      values.append(key)
      values.append(value)
  # 現在，您可以使用values列表查詢所有鍵值的值
  return values

In [4]:
def company_dataframe(values):
  # 建立空清單，將每個元組的數據存儲在清單中
  data = []

  # 遍歷清單中的每個元素
  for element in values:
      # 如果元素是元組，則加入清單中
      if isinstance(element, tuple):
          data.append(element)

  # 將清單轉換為DataFrame
  df = pd.DataFrame(data, columns=['id','name', 'exchange', 'industry', 'website'])
  return df

In [5]:
#欲查詢之股票
keys = input('輸入欲查詢之股票代號(用逗號分隔): ').split(',')

result = company_info() #return result
values = company_id(keys,result) # return values
df = company_dataframe(values) # return df
df

輸入欲查詢之股票代號(用逗號分隔): 2330


,id,name,exchange,industry,website
0,22099131,台積電,sii,半導體業,https://www.tsmc.com


### 原始爬蟲表格檢查

In [24]:
#@title  { display-mode: "form"  , run: "auto" }
#@markdown ###依 **產業別** 進行搜尋

market_input="sii" #@param ["sii","otc","rotc","pub"]
market=str(market_input)
res =requests.get(f"https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK={market}&step=1&firstin=1") #透過網址把抓到的資料放到res
df = pd.read_html(res.text)[0] #把需要的資料放在df
df.columns = df.columns.str.replace(' ', '') #將公司 代號，去除空格，成為公司代號
filt = df[df['公司代號']!= '公司 代號'] #利用欄位塞選，濾掉重複性的標題
filt.insert(1,'市場別',market)
filt

,公司代號,市場別,公司名稱,公司簡稱,產業類別,外國企業註冊地國,住址,營利事業統一編號,董事長,總經理,...,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,公司網址,投資人關係聯絡人,投資人關係聯絡人職稱,投資人關係聯絡電話,投資人關係聯絡電子郵件,公司網站內利害關係人專區網址
0,1101,sii,臺灣水泥股份有限公司,台泥,水泥工業,－,台北市中山北路2段113號,11913502,張安平,程耀輝,...,TCC,"No.113, Sec.2, Zhongshan N. Rd., Taipei City 1...",(02)2531-6529,finance@taiwancement.com,http://www.taiwancement.com,永續辦公室,永續辦公室,02-25317099,ir@taiwancement.com,http://www.taiwancement.com/tw/csr/csr5-1.html
1,1102,sii,亞洲水泥股份有限公司,亞泥,水泥工業,－,台北市大安區敦化南路2段207號30、31樓,03244509,徐旭東,李坤炎,...,ACC,"30-31F., No.207, Sec. 2, Dunhua S. Rd., Da' an...",02-2378-5191,service@acc.com.tw,www.acc.com.tw,陳韋仲,投資人關係副經理,02 27338000 ext.8336,ir@acc.com.tw,http://www.acc.com.tw/tw/stakeholder/stakehold...
2,1103,sii,嘉新水泥股份有限公司,嘉泥,水泥工業,－,台北市中山北路2段96號,11892801,張剛綸,王立心,...,CHC,"No.96, Sec. 2, Zhongshan N. Rd., Zhongshan Dis...",(02)2511-4002,ir@chcgroup.com.tw,www.chcgroup.com.tw,金哲毅,永續發展辦公室經理,(02)2551-5211#277,ir@chcgroup.com.tw,www.chcgroup.com.tw/index.php?route=system/sys...
3,1104,sii,環球水泥股份有限公司,環泥,水泥工業,－,台北市南京東路二段125號10樓,07568009,博智投資股份有限公司,侯智升,...,UCC,"10th F., 125 Nanking E.Rd Sec. 2, Taipei Taiwan",02-25075870,ucc@ucctw.com,www.ucctw.com,楊宗仁,財務長,02-2507-7801,p1590@ucctw.com,http://www.ucctw.com/m/412-1518-12822.php?Lang...
4,1108,sii,幸福水泥股份有限公司,幸福,水泥工業,－,台北市松江路237號15樓,40601248,陳兩傳,陳兩傳,...,LUCKY CEMENT,"15TH FL.,NO.237,SUNG KIANG RD., TAIPEI,TAIWAN,...",25048672,wendy@luckygrp.com.tw,www.luckygrp.com.tw,溫秀菊,財務部經理,25092188,wendy@luckygrp.com.tw,http://www.luckygrp.com.tw/tw/index.asp?au_id=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,9944,sii,新麗企業股份有限公司,新麗,其他,－,桃園市八德區聯華街41號,33375909,簡瑞瑞,簡榮富,...,Shinih,"No.41, Lianhua St Bade Dist., Taoyuan City 334...",(03)3653635,ir@shinih.com,www.shinih.com.tw,施中南,協理,03-3659903,ir@shinih.com,https://www.shinih.com/stakeholder
1053,9945,sii,潤泰創新國際股份有限公司,潤泰新,其他,－,台北市八德路二段三０八號十一樓之一,12139612,簡滄圳,李志宏,...,RUENTEX DEVELOP,"11/F,No.308,Section 2, Bade Road, Taipei,Taiwa...",81617968,cmc@mail.ruentex.com.tw,www.rt-develop.com.tw,葉天政,副總經理,81619888,Rt001374@mail.ruentex.com.tw,www.rt-develop.com.tw
1054,9946,sii,三發地產股份有限公司(原:金革),三發地產,建材營造,－,台北市松山區八德路3段30號12樓,84466749,鍾鼎晟,劉書宏(代),...,SANFAR,"12F, No.30, Section 3, Bade Road, Songshan Dis...",02-2570-6985,contact01@sanfar.com.tw,https://www.sanfar.com.tw,余賀婷,投資人關係,02-2570-9988,contact01@sanfar.com.tw,https://www.sanfar.com.tw/zh-TW/investor#inves...
1056,9955,sii,佳龍科技工程股份有限公司,佳龍,綠能環保,－,桃園市觀音區大潭里一鄰環科路323號,97211972,吳界欣,吳界欣,...,SDTI,"No.323,Huanke Rd.,Guanyin Dist., Taoyuan 328,T...",(03)473-5259,service@sdti.com.tw,www.sdti.com.tw,陸怡豪,策略顧問,(03)4736566,service@sdti.com.tw,https://www.sdti.com.tw/investors_shareholder....


In [25]:
#原始代碼-產生4種市場之公司基本資料
market=input("請輸入市場別: ") #sii=上市 otc=上櫃 rotc=興櫃 pub公開發行

res =requests.get(f"https://mops.twse.com.tw/mops/web/ajax_t51sb01?TYPEK={market}&step=1&firstin=1") #透過網址把抓到的資料放到res
df = pd.read_html(res.text)[0] #把需要的資料放在df
df.columns = df.columns.str.replace(' ', '') #將公司 代號，去除空格，成為公司代號
filt = df[df['公司代號']!= '公司 代號'] #利用欄位塞選，濾掉重複性的標題
filt.insert(1,'市場別',market)
# filt.set_index('公司代號', inplace=True) #將公司代號欄位，直接設計成index

# #把資料分別儲存至空變數中
# if market == "sii":
#   df = filt
# elif market == "otc":
#   df = filt
# elif market == "rotc":
#   df = filt
# else:
#   df = filt

# #建立空字典
# # result={}

# for index,col in df.iterrows():
#     #key: value1、value2...、value2...
#     result[col['公司代號']]=(col['營利事業統一編號'],col['公司簡稱'],col['市場別'],col['產業類別'],col['公司網址'])

filt #檢查1

KeyboardInterrupt: ignored

In [12]:
filt.head(20)

,公司代號,市場別,公司名稱,公司簡稱,產業類別,外國企業註冊地國,住址,營利事業統一編號,董事長,總經理,...,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,公司網址,投資人關係聯絡人,投資人關係聯絡人職稱,投資人關係聯絡電話,投資人關係聯絡電子郵件,公司網站內利害關係人專區網址
0,1101,sii,臺灣水泥股份有限公司,台泥,水泥工業,－,台北市中山北路2段113號,11913502,張安平,程耀輝,...,TCC,"No.113, Sec.2, Zhongshan N. Rd., Taipei City 1...",(02)2531-6529,finance@taiwancement.com,http://www.taiwancement.com,永續辦公室,永續辦公室,02-25317099,ir@taiwancement.com,http://www.taiwancement.com/tw/csr/csr5-1.html
1,1102,sii,亞洲水泥股份有限公司,亞泥,水泥工業,－,台北市大安區敦化南路2段207號30、31樓,03244509,徐旭東,李坤炎,...,ACC,"30-31F., No.207, Sec. 2, Dunhua S. Rd., Da' an...",02-2378-5191,service@acc.com.tw,www.acc.com.tw,陳韋仲,投資人關係副經理,02 27338000 ext.8336,ir@acc.com.tw,http://www.acc.com.tw/tw/stakeholder/stakehold...
2,1103,sii,嘉新水泥股份有限公司,嘉泥,水泥工業,－,台北市中山北路2段96號,11892801,張剛綸,王立心,...,CHC,"No.96, Sec. 2, Zhongshan N. Rd., Zhongshan Dis...",(02)2511-4002,ir@chcgroup.com.tw,www.chcgroup.com.tw,金哲毅,永續發展辦公室經理,(02)2551-5211#277,ir@chcgroup.com.tw,www.chcgroup.com.tw/index.php?route=system/sys...
3,1104,sii,環球水泥股份有限公司,環泥,水泥工業,－,台北市南京東路二段125號10樓,07568009,博智投資股份有限公司,侯智升,...,UCC,"10th F., 125 Nanking E.Rd Sec. 2, Taipei Taiwan",02-25075870,ucc@ucctw.com,www.ucctw.com,楊宗仁,財務長,02-2507-7801,p1590@ucctw.com,http://www.ucctw.com/m/412-1518-12822.php?Lang...
4,1108,sii,幸福水泥股份有限公司,幸福,水泥工業,－,台北市松江路237號15樓,40601248,陳兩傳,陳兩傳,...,LUCKY CEMENT,"15TH FL.,NO.237,SUNG KIANG RD., TAIPEI,TAIWAN,...",25048672,wendy@luckygrp.com.tw,www.luckygrp.com.tw,溫秀菊,財務部經理,25092188,wendy@luckygrp.com.tw,http://www.luckygrp.com.tw/tw/index.asp?au_id=...
5,1109,sii,信大水泥股份有限公司,信大,水泥工業,－,台北市寶慶路37號7樓,03279507,楊智雄,楊大寬,...,HsingTa,"7TH FL.,No.37 Pao Ching Road Taipei Taiwan R.O.C.",23314086,service@hsingta.com.tw,www.hsingta.com.tw,蔡素秋,財會部經理,02-23816731#160,emily@hsingta.com.tw,http://www.hsingta.com.tw/Aboutusdetail.asp?lv...
6,1110,sii,東南水泥股份有限公司,東泥,水泥工業,－,高雄市前金區五福三路21號4樓之1,83078600,陳敏斷,吳炎輝,...,TUNA CEMENT,"NO.21 Wu Fu 3 Road Kaohsiung, Taiwan, R.O.C.",07-9505895,southeast@secement.com,www.southeastcement.com.tw,黃薪翰,財務部副總經理,07-2711121,vance.huang@secement.com,http://www.southeastcement.com.tw/index.php?q=...
7,1201,sii,味全食品工業股份有限公司,味全,食品工業,－,台北市松江路125號,11347802,陳宏裕,張教華,...,WEI CHUAN,"10F., No.125, Sung Chiang Rd. Taipei, Taiwan, ...",(02)25063182,service@weichuan.com.tw,http://www.weichuan.com.tw,黃國禎,財務長,(02)25065020,KC_Huang@weichuan.com.tw,http://www.weichuan.com.tw/investor
8,1203,sii,味王股份有限公司,味王,食品工業,－,台北市中山北路二段79號5樓,07067309,陳清福,陳恭平,...,VE WONG,"5TH FL., 79 CHUNG SHAN N. RD., SEC.2 TAIPEI TA...",(02)25811006,wei@vewong.com.tw,http://www.vewong.com,魏璟雄,經理,(02)25717271-610,wei@vewong.com.tw,http://www.vewong.com/ec99/ushop20069/profile_...
9,1210,sii,大成長城企業股份有限公司,大成,食品工業,－,台南市永康區蔦松二街三號,73008303,韓家宇,韓芳豪,...,GREATWALL,3 NIAOSUN 2TH STREET YONGKONG TAINAN CONTY TAIWAN,06-2534596,ir@ms.greatwall.com.tw,http://www.dachan.com,周叔恆,特助,02-26577111,ir@ms.greatwall.com.tw,https://www.dachan.com/forward4?subChannelId=7...


In [17]:
#csv下載
filt.to_csv('oct_230901.csv',index=False)